---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [7]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [8]:
def get_list_of_university_towns():
    # Let's create a new variable for all the values
    towns = []
    # Read the file
    with open('university_towns.txt', 'r+') as file:
        state = "Unknown"
        # Read the 
        for line in file:
            if "[edit]" in line:
                state = line.split("[")[0].strip()
            else:
                # Get the country and store into the Dictionary for the dataframe
                towns.append([state, line.split("(")[0].strip()])
    # Create a dataframe using the data read
    return pd.DataFrame(towns, columns=["State", "RegionName"])

In [8]:
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [7]:
## Recession definition
def check_reccession(row):
    # By default the recesion is set to False
    result = False
    if (row["i-3"] < row["i-4"]) and (row["i-2"] < row["i-3"]):
        result = True
    # Set the final result 
    row["recession"] = result
    return row

def get_GDP():
    # Open the Excel file
    excel = pd.ExcelFile("gdplev.xls")
    # Form the first sheet only take the columns E and G with the quarter and chained GDP 
    GDP = excel.parse(excel.sheet_names[0],names=["QuarterYear","GDP"],skiprows=7, parse_cols="E,G")
    # Split the columns quarter year into year
    GDP["Year"] = GDP["QuarterYear"].map(lambda cell: cell.split("q")[0])
    # Convert to numeric (floa, integer or wathever) values from columns Year and Quarter
    GDP["Year"] = pd.to_numeric(GDP["Year"])
    GDP["GDP"] = pd.to_numeric(GDP["GDP"])
    # Filter the values from 2000 (we also need previous year to compute the recession for the first year)
    GDP = GDP[GDP["Year"] >= 1999]
    # Reset the index and delete the old one.
    GDP.reset_index(drop=True,inplace=True)
    # Create new columns with the last 4 quarters in order to cumpute the reccesion
    columns = ["i-4","i-3","i-2","i-1"]
    values = [[np.NaN,np.NaN,np.NaN,np.NaN]] * (len(GDP["GDP"]) )
    new_columns = pd.DataFrame(values, columns = columns)
    # Add to the columns the data from GDP
    for i in range(4, len(GDP["GDP"])):
        new_columns.ix[i, columns] = GDP["GDP"].iloc[i-4:i].values  
    #Merge the new columns with the GDO set from 2000	
    GDP = pd.concat([GDP,new_columns], axis = 1 )
    #Once both set have been merged, et the index
    GDP.set_index("QuarterYear", inplace=True)
    ## Let's create a new column with recession periods
    GDP["recession"] = False
    # Apply recession function to the dataframe by row
    return GDP.apply(check_reccession,axis = 1)

In [13]:
def get_recession_start():
    GDP = get_GDP()
    recGDP = GDP[GDP["recession"]==True]
    bottom = GDP.index.get_loc(recGDP["GDP"].argmin())
    return GDP.index[bottom- 3]

In [9]:
def get_recession_end():
    GDP = get_GDP()
    recGDP = GDP[GDP["recession"]==True]
    return recGDP["GDP"].argmax()

In [12]:
def get_recession_bottom():
    GDP = get_GDP()
    recGDP = GDP[GDP["recession"]==True]
    return recGDP["GDP"].argmin()

In [10]:
def convert_housing_data_to_quarters():
    homes = pd.read_csv("City_Zhvi_AllHomes.csv")
    # Apply the ransfomration to moths to quarters
    hquarters = homes.loc[:,["RegionName","State"]]
    # Let's loop from 2000 to 2016
    for year in range(2000,2017):
        # Let's loop over the four quarters per year
        for quarter in range(0,4):
            #Let's loop over the 12/4 = 3 mothns per quarter
            columns = [str(year)+"-"+str((quarter * 3) + month).zfill(2) for month in range(1,4)]
            # Check if this quarter need to be added
            if (any((True for x in homes.columns.values if x in columns))):
                #Since get doesn't seems to work fine it the columns doesn't exist, check if a column need
                #to be added
                for column in columns:
                    if not column in homes.columns.values:
                        #Add the column with NaN values
                        homes[column] = np.nan
                hquarters[str(year) + "q" + str(quarter+1)] = homes.get(columns).mean(skipna=True, axis=1)
    # Map the states 
    hquarters["State"] = hquarters["State"].map(states)
    #Set the new idnex
    return hquarters.set_index(["State","RegionName"])

In [14]:
convert_housing_data_to_quarters().shape

(10730, 67)

In [15]:
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

202266.66666666666

In [11]:
def run_ttest():
    #Let's get the columns where the recession star and the bottom
    fromDate = get_recession_start()
    toDate = get_recession_end()
    # Get the housing data and the university towns
    data = convert_housing_data_to_quarters()
    towns = get_list_of_university_towns()
    #Let's use the same index as the data for the university Towns dataframe
    towns.set_index(["State","RegionName"],inplace = True)
    #Compute the ratio housing prices from the bottom to the start recession period
    data = data.loc[:, fromDate:toDate]
    data["ratio"] = data.loc[:, fromDate] / data.loc[:, toDate] 
    #Get the universisty towns
    university = pd.merge(data,towns,how='inner',left_index=True,right_index=True)
    #Left outter join
    non_university = pd.merge(data, towns,how='outer',left_index=True, right_index=True, indicator=True)
    non_university = non_university[non_university._merge == 'left_only']
    #Call to ttest                                
    statistic, p_value = tuple(ttest_ind(university["ratio"].dropna(), non_university["ratio"].dropna()))
    outcome = statistic < 0
    different = p_value < 0.05
    better = ["non-university town", "university town"]
    return (different, p_value, better[outcome])

In [12]:
run_ttest()

NameError: name 'pd' is not defined